In [ ]:
# Energy Infrastrcuture 
# Course Asssignment Part 2 
# Candidate Number: 1059922

In [ ]:
# Step 1: Insertion of libraries and creation of an empty network 
import pandapower as pp
import pandapower.plotting as pplot
import numpy as np

net = pp.create_empty_network()

In [ ]:
# STEP 2: Creation of buses, loads and transformers 

HV=132 # high Voltage (kV)
MV=11 # medium voltage (kV)
LV=0.415 # low voltage (kV)
PV_gen=0.0034 # peak solar power capacity per household (MW)
ADMD=0.004 # ADMD per house (MW) 


# bus 0 is connected to the external grid power source
#b0 and b1 are the buses on each side of the 3MVA 132/11 kV transformer 

b0 = pp.create_bus(net, vn_kv = HV,  name="Power Source")
b1 = pp.create_bus(net, vn_kv = MV,  name="Bus 1") 

# The 4 buses just before each of the 0.5MVA 11/0.415 kV transformer are medium voltage (11kV) buses named b2,b3,b4 and b5
for i in range(4):
    pp.create_bus(net, vn_kv = MV,  name=f"Bus {i+2}")

# The 4 buses after each of the 0.5MVA 11/0.415 kV transformer are Low voltage (0.415kV) buses named b6,b7,b8 and b9
for i in range(4):
    pp.create_bus(net, vn_kv = LV,  name=f"Bus {i+6}")



# the 168 houses are divided into 4 sets (A-B-C-D) each group is composed by 2 groups of 7
# The 168 houses are connected into 3-phase "Wye" connections
# Each of the 4 sets is made up of 14 3-house connections. 
# The buses for each of the 4 sets are named A1,A2,.....A14,B1,B2,.....,D14

# Group A
#10-23
for i in range(14):
    pp.create_bus(net, vn_kv=LV, name=f"Bus A{i+1}") 

# Group B
#24-37
for i in range(14):
    pp.create_bus(net, vn_kv=LV, name=f"Bus B{i+1}") 

# Group C
#38-51
for i in range(14):
    pp.create_bus(net, vn_kv=LV, name=f"Bus C{i+1}") 

# Group D
#52-65
for i in range(14):
    pp.create_bus(net, vn_kv=LV, name=f"Bus D{i+1}") 

net.bus

In [ ]:
# STEP 3: Creating the external grid
pp.create_ext_grid(net, bus=b0, vm_pu=1.05, name="Slack Bus")

In [ ]:
# STEP 4: Creating the transformers 

# all 5 transformers are provided by Wilson Power Solutions Cables

# The first transformer is a 3MVA 132/11kV transformer between b0 and b1 
sn_MV= 3 # Rated appearent power  (MVA)
vkr_MV=1 # Real part of relative short-cicruit voltage
vk_MV=10 # Relative short-circuit voltage
pfe_MV=5 #iron losses
i0_MV=5 #open loop losses in percent of rated current 

pp.create_transformer_from_parameters(net, hv_bus=b0, lv_bus=b1, sn_mva=sn_MV, vn_hv_kv=HV, vn_lv_kv=MV, vkr_percent=vkr_MV, vk_percent=vk_MV, pfe_kw=pfe_MV, i0_percent=i0_MV,name="Trafo 1")

#There are 4 transformers for each of the 4 main roads alleys and placed in the middle of the sets
# These 4 transformers are 0.5 MVA transformers 11/0.415 kV between each of b2,b3,b4,b5 and b6,b7,b8,b9 respectivelly 

sn_LV=0.5 # Rated appearent power  (MVA)
vkr_LV=1 # Real part of relative short-cicruit voltage
vk_LV=10 # Relative short-circuit voltage
pfe_LV=1.2 #iron losses 
i0_LV=5 #open loop losses in percent of rated current 

for i in range (4):
    pp.create_transformer_from_parameters(net, hv_bus=i+2, lv_bus=i+6, sn_mva=sn_LV, vn_hv_kv=MV, vn_lv_kv=LV, vkr_percent=vkr_LV, vk_percent=vk_LV, pfe_kw=pfe_LV, i0_percent=i0_LV,name=f"Trafo {i+2}")


net.trafo

In [ ]:
# Solar Switch 

# By turning ON (1) the PV panels are activated and additional loads are assigned to each bus, this represents the summer scenario
# By turning OFF (0) the PV panels are not used, this represents the winter scenario

solar = 0 # OFF=0 or ON=1

In [ ]:
# STEP 5: Creating the loads and dummy buses 

# Assuming a power factor of 0.9 (pf=0.9)
# theta=ArcCos(pf)=ArcCos(0.9)=0.451
#Reactive Power (VAR) = Q = P*tan(theta)
#tan(theta)=0.482
tan=0.482



# The power on each bus is different depending on the seosons, as during the winter no solar power is used and the houses receive all of their power from the substation 
# While in the summer, the houses generate power from the PV's on the roofs as well as from the substation. At each bus , a negative load is created due to solar not able to store energy
# 14 buses per 42 houses, 3 houses per bus

if solar==0:
    for i in range (14): # creating loads for the Group A buses
        pp.create_load(net, bus=i+10, p_mw=ADMD*3, q_mvar=ADMD*3*tan, name=f"Bus A{i+1}")
    for i in range (14): # creating loads for the Group B buses
        pp.create_load(net, bus=i+24, p_mw=ADMD*3, q_mvar=ADMD*3*tan, name=f"Bus B{i+1}")
    for i in range (14): # creating loads for the Group C buses
        pp.create_load(net, bus=i+38, p_mw=ADMD*3, q_mvar=ADMD*3*tan, name=f"Bus C{i+1}")    
    for i in range (14): # creating loads for the Group D buses
        pp.create_load(net, bus=i+52, p_mw=ADMD*3, q_mvar=ADMD*3*tan, name=f"Bus D{i+1}")
    
else:
    for i in range (14): # creating loads for the Group A buses
        pp.create_load(net, bus=i+10, p_mw=ADMD*3*0.9, q_mvar=ADMD*3*tan*0.9, name=f"Bus A{i+1}")
        pp.create_load(net, bus=i+10, p_mw=-PV_gen*3*0.21, q_mvar=-PV_gen*3*tan*0.21, name=f"Bus A{i+1}+PV")
    
    for i in range (14): # creating loads for the Group B buses
        pp.create_load(net, bus=i+24, p_mw=ADMD*3*0.9, q_mvar=ADMD*3*tan*0.9, name=f"Bus B{i+1}")
        pp.create_load(net, bus=i+24, p_mw=-PV_gen*3*0.21, q_mvar=-PV_gen*3*tan*0.21, name=f"Bus B{i+1}+PV")
    
    for i in range (14): # creating loads for the Group C buses
        pp.create_load(net, bus=i+38, p_mw=ADMD*3*0.9, q_mvar=ADMD*3*tan*0.9, name=f"Bus C{i+1}")
        pp.create_load(net, bus=i+38, p_mw=-PV_gen*3*0.21, q_mvar=-PV_gen*3*tan*0.21, name=f"Bus C{i+1}+PV")
    
    for i in range (14): # creating loads for the Group D buses
        pp.create_load(net, bus=i+52, p_mw=ADMD*3*0.9, q_mvar=ADMD*3*tan*0.9, name=f"Bus D{i+1}")
        pp.create_load(net, bus=i+52, p_mw=-PV_gen*3*0.21, q_mvar=-PV_gen*3*tan*0.21, name=f"Bus D{i+1}+PV")
    

    
    
net.load

In [ ]:
# STEP 6: Creating the transmission lines connecting the external grid, transformers and buses

# total of 56 LV lines from the 4 transformers to each bus representing the 3-houses groupings
# total of 4 lines from the main MV bus (b1) to the other MV buses on one side of the transformers (b2,b3,b4,b5)
# lines between the buses surrounding the transformers are assumed neglegible
# Assumed nominally buried 


# this array represents the distances between bus 1 (after the first transformer) and each of the 4 MV buses before the 11/0.415kV transformers (b2,b3,b4,b5)
#On the design layout these lines are in blue
# all length are in km 
a = np.array([0.620,0.390,0.390,0.620],dtype=float)

#this array represents the lengths in order of the lines from the LV buses (b6,b7,b8,b9) at the other side  of the 11/0.415kV transformer to buses associated with the plots of land
#On the design layout these lines are in green
# all length are in km 
b = np.array([0.280,0.230,0.130,0.08,0.08,0.230,0.230,0.330,0.180,0.180,0.03,0.130,0.180,0.280],dtype=float)

#Line values for medium voltage buses from https://www.elandcables.com/media/1460428/enw-6-35-11-12-kv-single-core-triplex-unarmoured-aluminium-conductors.pdf?fbclid=IwAR2TY0KcL_4W3ATMvPCJZg1PkAzUkArabDpv_krMc7ltxj2w10yUA9xNg8g
# I = ADMD*No. houses / (pf * root three * Voltge)= 4*42/(0.9*11*√3) =9.79A
# Area of the 3 core cable = 95 mm2 
r_m_MV=0.411 # line resistance in ohm per km
x_m_MV=0.131 # line reactance in ohm per km
c_f_MV=1e-9 # line capacitance in nano farad per km 
maxka_MV=0.231 # maximum thermal current in kA

# Lines from bus 1 to the other 4 MV buses before the transformers

for i in range (4):
    pp.create_line_from_parameters(net,from_bus=b1, to_bus=i+2,length_km=a[i], r_ohm_per_km = r_m_MV, x_ohm_per_km = x_m_MV, c_nf_per_km = c_f_MV, max_i_ka=maxka_MV)

# I = ADMD*No. houses / (pf * root three * Voltge)= 4*3/(0.9*0.415*√3) = 18.5 A
# sizing for 4 mm2 https://www.elandcables.com/media/3ljpmrxt/bs-5467-copper-conductor-multi-core-swa-pvc-basec-0-6-1kv-cable.pdf
r_m_LV=16e-3 # line resistance in ohm per km
x_m_LV=0 # line reactance in ohm per km  
c_f_LV=1e-9 # line capacitance in nano farad per km
maxka_LV=0.028 # maximum thermal current in kA


for i in range(14): 
      pp.create_line_from_parameters(net,from_bus=6, to_bus=i+10, length_km=b[i], r_ohm_per_km = r_m_LV, x_ohm_per_km = x_m_LV, c_nf_per_km = c_f_LV, max_i_ka=maxka_LV)
    
for i in range(14): 
      pp.create_line_from_parameters(net,from_bus=7, to_bus=i+24, length_km=b[i], r_ohm_per_km = r_m_LV, x_ohm_per_km = x_m_LV, c_nf_per_km = c_f_LV, max_i_ka=maxka_LV)
        
for i in range(14): 
      pp.create_line_from_parameters(net,from_bus=8, to_bus=i+38, length_km=b[i], r_ohm_per_km = r_m_LV, x_ohm_per_km = x_m_LV, c_nf_per_km = c_f_LV, max_i_ka=maxka_LV)

for i in range(14): 
      pp.create_line_from_parameters(net,from_bus=9, to_bus=i+52, length_km=b[i], r_ohm_per_km = r_m_LV, x_ohm_per_km = x_m_LV, c_nf_per_km = c_f_LV, max_i_ka=maxka_LV)
        
net.line

In [ ]:
# power flow analysis 
pp.runpp(net, algorithm='nr');

In [ ]:
net.res_bus

In [ ]:
net.res_trafo

In [ ]:
net.res_line

In [ ]:
net.res_load

In [ ]:
#pp.to_excel(net, 'prob2results_YES_SOLAR.xlsx')
#pp.to_excel(net, 'prob2results_NO_SOLAR.xlsx')